In [1]:
from param import Param
from sde import *
from fparam import *
import numpy as np
import math
from matplotlib import pyplot as plt

In [ ]:
# First we calibrate using the heston model
params = {
    "h_0"  : Param(.11/math.sqrt(21)),
    "theta": Param(),
    "rho"  : Param(),
    "kappa": Param(),
    "xi"   : Param(),
    "dt"   : Param(1/252),
    "r"    : Param(0.0411/252),
    "S_0"  : Param(100)
}

# Model functions
def feller(params):
    k     = params["kappa"].eval()
    theta = params["theta"].eval()
    xi    = params["xi"].eval()
    return 2 * k * theta > xi * xi

def mu_h(h_t, t, params=None):
    k = params["kappa"].eval()
    theta = params["theta"].eval()
    return k * (theta - h_t)

def v_h(h_t, t, params=None):
    xi = params["xi"].eval()
    return xi * math.sqrt(max(h_t, 0.))

def mu(S_t, t, params=None):
    r = params["r"].eval()
    return r * S_t

def v(S_t, t, cir, params=None):
    vol = cir.nodes[t-1].eval()
    return math.sqrt(max(vol, 0.)) * S_t

drift_h  = FParam(mu_h, params)
vol_h    = FParam(v_h, params)
T   = 252
N   = 10000

cir_sim = EulerSimulation(drift_h, vol_h, params["h_0"], T, params["dt"], N)

drift_s = FParam(mu, params)
vol_s   = FParam(v, params)

simulation = StochVolSimulation(drift_s, vol_s, params["S_0"], cir_sim, T, params["dt"], N, rho=params["rho"].eval())

In [ ]:
def plot_sims(cir_sim, simulation):
    for i in range(N):
        plt.plot(cir_sim.results[:, i])
    plt.xlabel("Timesteps (252 total)")
    plt.ylabel("Daily Volatility")
    plt.title("Volatility Simulation")
    plt.show()

    for i in range(N):
        plt.plot(simulation.results[:, i])
    plt.xlabel("Timesteps (252 total)")
    plt.ylabel("Index Price")
    plt.title("Index Price Simulation")
    plt.show()

def run_sims(cir_sim, simulation):
    cir_sim.run_all()
    simulation.run_all()